In [ ]:
#Load a bunch of modules

import numpy as np
import pandas as pd
import geopandas as gpd
import streamlit as st
import folium as fl
from folium.plugins import FastMarkerCluster,MarkerCluster,MiniMap,HeatMap
import streamlit_folium as sf
import branca.colormap as cm
import os as os
#import wget
import tarfile
import datetime as dt
import matplotlib.pyplot as plt
#import Iframe
import base64
import re

print('loaded 9 modules')

In [ ]:
 #Make sure we're in the correct direcotyr

os.chdir('/Users/arbetter/Coding/Streamlit/CAwellsAg')

In [ ]:
pd.options.display.max_columns=None
pd.options.display.max_rows=None

#streamlit message size
MESSAGE_SIZE_LIMIT = 1000.*int(1e6) #1000 MB

In [ ]:
wells_gdf = gpd.read_file('AllWells_gis/Wells_All.shp')

wells_gdf['Wells']=1

print(wells_gdf.crs)


In [ ]:
len(wells_gdf)

In [ ]:
wells_gdf.to_crs('EPSG:4269',inplace=True)

wells_gdf.crs

In [ ]:
countywells_df = wells_gdf.groupby('CountyName').sum()

countywells_df

In [ ]:
# Read in California counties

ca_counties_gdf = gpd.read_file("geodata/california_counties/CaliforniaCounties.shp")

ca_counties_gdf.fillna(0,inplace=True)

ca_counties_gdf['Wells']=0

ca_counties_gdf.crs



In [ ]:
ca_counties_gdf.to_crs('EPSG:4269',inplace=True)

In [ ]:
for county_loc in countywells_df.index:
    c1 = county_loc
#    print(county_loc,c1)

    # add Offshore wells to their respective counties
    c2 = c1.replace("  "," ")
    c3 = c2.replace("Offshore","").strip()
    
    # same with Island, except Santa Cruz Island is in Santa Barbara County
    if (c3 == 'Santa Cruz Island'):
        c = 'Santa Barbara'
    else:
        c = c3.replace("Island","").strip()
    
#    print(c1,c2,c3,c)

# put islands back in their counties
    if (c == 'San Miguel'):
        c = 'Santa Barbara'
    elif (c == "San Nicolas"):
        c = 'Ventura'
    elif (c == 'Santa Rosa'):
        c = 'Santa Barbara'
    
    print(county_loc,c)
    
    try:
        icounty = ca_counties_gdf[ca_counties_gdf['NAME']==c].index[0]
        ca_counties_gdf.at[icounty,'Wells']=countywells_df.at[c,'Wells']+ca_counties_gdf.at[icounty,'Wells']
    except:
        print(c,'not found')
        
        


In [ ]:
ca_counties_gdf = ca_counties_gdf[['NAME','Wells','STATE_NAME','POP2010','POP10_SQMI','SQMI','CountyFIPS','geometry']]

ca_counties_gdf.head()

In [ ]:
ca_counties_gdf.crs

In [ ]:
fields_gdf = gpd.read_file('Field_Boundary/Field_Boundary.shp')

fields_gdf.crs

In [ ]:
fields_gdf.to_crs('EPSG:4269',inplace=True)

print(fields_gdf.crs)

In [ ]:
fields_gdf.head()

In [ ]:
facilities_gdf = gpd.read_file('FacilityBoundary/FacilityBoundary_20200529.shp')

facilities_gdf.crs



In [ ]:
facilities_gdf.to_crs('EPSG:4269',inplace=True)

facilities_gdf.crs

In [ ]:
facilities_gdf.head()

In [ ]:
socal_counties_gdf = gpd.GeoDataFrame(columns = ca_counties_gdf.columns)

socal_counties_gdf.crs

In [ ]:
socal_counties = ['San Luis Obispo','Santa Barbara','Kern','Ventura','Los Angeles','Orange','San Bernardino',
                   'Riverside','San Diego','Imperial']

socal_counties = ['Kern']

#socal_counties = ['Ventura']

#socal_counties = ['Los Angeles']

for county_loc in ca_counties_gdf['NAME']:

    if (county_loc in socal_counties):
#        entry = ca_counties_gdf.loc[ca_counties_gdf['NAME'] == county]
#        print (entry)
#        socal_counties_gdf = socal_counties_gdf.append(entry)
        socal_counties_gdf = socal_counties_gdf.append(ca_counties_gdf.loc[ca_counties_gdf['NAME'] == county_loc])
#    else:
#        print('skipped',county)
        
socal_counties_gdf.plot()

In [ ]:
county_latlon = {}
county_latlon['Kern'] = [35.34,-118.2]
county_latlon['SoCal'] = [34.1, -117.3]
county_latlon['Ventura'] = [34.27, -119.2]
county_latlon['Los Angeles'] = [34,-118.25]

county = socal_counties[0]

county

In [ ]:
socal_counties_gdf

In [ ]:
print(socal_counties_gdf.crs)

In [ ]:

                           
socal_counties_gdf.set_crs('EPSG:4269',inplace=True)

socal_counties_gdf.crs


In [ ]:
socal_counties_gdf

In [ ]:
fields_gdf = gpd.clip(fields_gdf,socal_counties_gdf,keep_geom_type=False)

fields_gdf.head()

In [ ]:
fields_gdf.crs

In [ ]:
facilities_gdf = gpd.clip(facilities_gdf,socal_counties_gdf,keep_geom_type=True)

facilities_gdf.head()

In [ ]:
facilities_gdf.crs

In [ ]:
canals_gdf = gpd.read_file('Canals_and_Aqueducts_Local/Canals_and_Aqueducts_Local.shp')

#canals_gdf.set_crs('EPSG:3310',inplace=True)


canals_gdf.crs

In [ ]:

canals_gdf.to_crs('EPSG:4269',inplace=True)

canals_gdf.crs

In [ ]:
canals_gdf = gpd.clip(canals_gdf,socal_counties_gdf,keep_geom_type=True)

canals_gdf.head()

In [ ]:
wells_gdf = gpd.clip(wells_gdf,socal_counties_gdf,keep_geom_type=True)

len(wells_gdf)

In [ ]:
wells_gdf.reset_index(inplace=True)

wells_gdf.head(10)

In [ ]:
# create latlon list for FastMarkerCluster

wells_latlon = []

for i in wells_gdf.index:
    
    latlon = []
    latlon.append(wells_gdf.at[i,'Latitude'])
    latlon.append(wells_gdf.at[i,'Longitude'])
    wells_latlon.append(latlon)
    
    

In [ ]:
landuse_gdf = gpd.read_file('i15_crop_mapping_2018_shp/i15_Crop_Mapping_2018.shp')

#landuse_gdf.fillna(0,inplace=True)

landuse_gdf.crs



In [ ]:
landuse_gdf['Crop'] = ''

landuse_gdf['Use_Index'] = ''

landuse_gdf = landuse_gdf[['UniqueID','SYMB_CLASS','MULTIUSE','SUBCLASS2','COUNTY','geometry','Crop','Use_Index']]

#landuse_gdf = gpd.clip(landuse_gdf,socal_counties_gdf,keep_geom_type=True)

# remove data that's not in SoCal; this is faster
for county_loc in set(landuse_gdf['COUNTY']):

    if county_loc not in socal_counties:
        print('NoCal',county_loc)
        landuse_gdf.drop(landuse_gdf.index[landuse_gdf['COUNTY'] == county_loc], inplace = True)
    else:
        print('SoCal',county_loc)

landuse_gdf.head(10)

In [ ]:
landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index)),'Use_Index']= 0

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index)),'Use_Index']= 1

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index)),'Use_Index']= 2

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='G'].index)),'Use_Index']= 3

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='I'].index)),'Use_Index']= 4

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index)),'Use_Index']= 5

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='R'].index)),'Use_Index']= 6

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index)),'Use_Index']= 7

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='U'].index)),'Use_Index']= 8

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='V'].index)),'Use_Index']= 9

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='X'].index)),'Use_Index']= 10

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='YP'].index)),'Use_Index']= 11



In [ ]:
landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='G'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Barley'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='G'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'Wheat'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='G'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='3'].index)),'Crop'] = 'Oats'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='G'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='6'].index)),'Crop'] = 'Misc Grain & Hay'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='G'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='7'].index)),'Crop'] = 'Mixed Grain & Hay'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='G'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Grain & Hay Crops (not specified)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='R'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Rice'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='R'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'Wild Rice'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='R'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Rice (not specified)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Cotton'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'Safflower'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='3'].index)),'Crop'] = 'Flax'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='4'].index)),'Crop'] = 'Hops'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='5'].index)),'Crop'] = 'Sugar Beets'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='6'].index)),'Crop'] = 'Corn (Field & Sweet)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='7'].index)),'Crop'] = 'Grain Sorghum'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='8'].index)),'Crop'] = 'Sudan'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='9'].index)),'Crop'] = 'Castor Beans'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='10'].index)),'Crop'] = 'Beans (dry)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='11'].index)),'Crop'] = 'Misc Field'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='12'].index)),'Crop'] = 'Sunflowers'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='13'].index)),'Crop'] = 'Hybrid sorghum/sudan'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='14'].index)),'Crop'] = 'Millet'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='15'].index)),'Crop'] = 'Sugar Cane'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='16'].index)),'Crop'] = 'Corn, Sorghum, or Sudan (grouped for remote sensing only)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='F'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Field Crop (not specified)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Alfalfa & Alfalfa Mixtures'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'Clover'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='3'].index)),'Crop'] = 'Mixed Pasture'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='4'].index)),'Crop'] = 'Native Pasture'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='5'].index)),'Crop'] = 'Induced High Water Table native Pasture'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='6'].index)),'Crop'] = 'Misc Grasses'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='7'].index)),'Crop'] = 'Turf Farms'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='8'].index)),'Crop'] = 'Bermuda Grass'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='9'].index)),'Crop'] = 'Rye Grass'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='10'].index)),'Crop'] = 'Klein Grass'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='P'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Pasture (not specified)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Artichokes'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'Asparagus'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='3'].index)),'Crop'] = 'Beans (green)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='4'].index)),'Crop'] = 'Cole Crops'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='5'].index)),'Crop'] = 'Not Listed'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='6'].index)),'Crop'] = 'Carrots'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='7'].index)),'Crop'] = 'Celery'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='8'].index)),'Crop'] = 'Lettuce (all types)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='9'].index)),'Crop'] = 'Melons, Squash, and Cucumbers (all types)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='10'].index)),'Crop'] = 'Onions and Garlic'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='11'].index)),'Crop'] = 'Peas'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='12'].index)),'Crop'] = 'Potatoes'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='13'].index)),'Crop'] = 'Sweet Potatoes'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='14'].index)),'Crop'] = 'Spinach'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='15'].index)),'Crop'] = 'Tomatoes (Processing)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='16'].index)),'Crop'] = 'Flowers, Nursery, and Christmas Tree Farms'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='17'].index)),'Crop'] = 'Mixed (4 or more)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='18'].index)),'Crop'] = 'Misc Truck'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='19'].index)),'Crop'] = 'Bush Berries'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='20'].index)),'Crop'] = 'Strawberries'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='21'].index)),'Crop'] = 'Peppers (Chili, Bell, etc)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='22'].index)),'Crop'] = 'Brocolli'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='23'].index)),'Crop'] = 'Cabbage'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='24'].index)),'Crop'] = 'Cauliflower'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='25'].index)),'Crop'] = 'Brussels Sprouts'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='26'].index)),'Crop'] = 'Tomatoes (market)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='27'].index)),'Crop'] = 'Greenhouse'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='28'].index)),'Crop'] = 'Blueberries'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='29'].index)),'Crop'] = 'Asian Leafy Vegetables'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='30'].index)),'Crop'] = 'Lettuce or Leafy Greeens (grouped for remote sensing only)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='31'].index)),'Crop'] = 'Potato or Sweet Potato (grouped for remote sensing only)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='T'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Truck, Nursery, or Berry Crops (unspecified)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Apples'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'Apricots'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='3'].index)),'Crop'] = 'Cherries'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='5'].index)),'Crop'] = 'Peaches & Nectarines'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='6'].index)),'Crop'] = 'Pears'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='7'].index)),'Crop'] = 'Plums'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='8'].index)),'Crop'] = 'Prunes'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='9'].index)),'Crop'] = 'Figs'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='10'].index)),'Crop'] = 'Misc Deciduous'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='11'].index)),'Crop'] = 'Mixed Deciduous'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='12'].index)),'Crop'] = 'Almonds'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='13'].index)),'Crop'] = 'Walnuts'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='14'].index)),'Crop'] = 'Pistachios'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='15'].index)),'Crop'] = 'Pomegranates'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='16'].index)),'Crop'] = 'Plums, Prunes, or Apricots (grouped for remote sensing only)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='D'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Deciduos Fruits and Nuts (unspecified)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Grapefruit'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'Lemons'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='3'].index)),'Crop'] = 'Oranges'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='4'].index)),'Crop'] = 'Dates'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='5'].index)),'Crop'] = 'Avocados'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='6'].index)),'Crop'] = 'Olives'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='7'].index)),'Crop'] = 'Misc Subtropical Fruit'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='8'].index)),'Crop'] = 'Kiwis'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='9'].index)),'Crop'] = 'Jojoba'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='10'].index)),'Crop'] = 'Eucalyptus'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='11'].index)),'Crop'] = 'Mixed Tropical Fruits'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='C'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Citrus & Tropical (unspecified)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='V'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Table Grapes'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='V'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'Wine Grapes'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='V'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='3'].index)),'Crop'] = 'Raisin Grapes'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='V'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='4'].index)),'Crop'] = 'Vineyards (unspecified)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='YP'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Young Perennial'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='I'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Land not cropped the current or previous crop season'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='I'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'New Lands being prepared for crop production'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='I'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='4'].index)),'Crop'] = 'Long Term, land consistently idle for 4 or more years'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='I'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Idle (not specified)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='S'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='1'].index)),'Crop'] = 'Farmsteads (includes a farm residence)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='S'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='2'].index)),'Crop'] = 'Livestock feed lot operations'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='S'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='3'].index)),'Crop'] = 'Dairies'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='S'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='4'].index)),'Crop'] = 'Poultry Farms'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='S'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='5'].index)),'Crop'] = 'Farmsteads (without a farm residence)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='S'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Miscellaneous semi-agricultural (small roads, ditches, non-planted areas of cropped fields)'

landuse_gdf.at[sorted(set(landuse_gdf[landuse_gdf['SYMB_CLASS']=='U'].index).intersection(landuse_gdf[landuse_gdf['SUBCLASS2']=='**'].index)),'Crop'] = 'Urban'
















In [ ]:
landuse_gdf['SYMB_CLASS'].replace('^G','Grain & Hay',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^R','Rice',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^F','Field Crops',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^P','Pasture',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^T','Truck, Nursery, & Berry Crops',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^D','Deciduous Fruits & Nuts',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^C','Citrus & Subtropical',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^V','Vineyards',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^I','Idle',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^S','Semi-Agricultural & Incidental to Agriculture',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^U','Urban',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^X','Unclassified Fallow',regex=True,inplace=True)
landuse_gdf['SYMB_CLASS'].replace('^YP','Young Perennial',regex=True,inplace=True)

In [ ]:
landtype_gdf = landuse_gdf.dissolve(by='Use_Index',as_index=False,sort=False)

landtype_gdf

In [ ]:
landuse_gdf = landuse_gdf[['SYMB_CLASS','Use_Index','Crop','COUNTY','geometry']]

landuse_gdf.head()

In [ ]:
colormap_landuse = cm.linear.Accent_08.scale(0,11).to_step(12)
colormap_landuse.caption = 'Land Use'

colormap_landuse

In [ ]:
colormap_wells = cm.linear.Set1_09.scale(0,75000)
colormap_wells.caption ='Wells per County'

colormap_wells

In [ ]:
#Create a b/w map of Southern California

#california_canalmap = fl.Map(location=[34.1,-117.3],
#                zoom_start=7,tiles=None)

#county is set above 

california_canalmap = fl.Map(location=county_latlon[county],
                zoom_start=8,tiles=None)

fl.TileLayer('cartodbpositron',name='BackGround',control=False).add_to(california_canalmap)




In [ ]:
county_latlon['Los Angeles']

In [ ]:
#Add well data
#fl.GeoJson(ca_counties_gdf, name='Wells per County', control=True,


#fl.GeoJson(socal_counties_gdf, name='Wells per County', control=True,show=False,
#           style_function = lambda x: {"weight":1.5, 
#                                       'color':'grey',
##                                       'fillColor':'transparent',
#                            'fillColor':colormap_wells(x['properties']['Wells']), 
#                            'fillOpacity':0.75
#               },
           
#               highlight_function = lambda x: {'fillColor': '#000000', 
#                                'color':'#000000', 
#                                'fillOpacity': 0.5, 
#                                'weight': 0.1
#               },
               
#               tooltip=fl.GeoJsonTooltip(
#                   fields=['NAME','Wells'],
#                   aliases=['County:','Wells:'],
#                   labels=True,
#                   localize=True
#               ),
#               ).add_to(california_canalmap)


In [ ]:
# Add field data
fl.GeoJson(fields_gdf, name='Fields', control=True,

           style_function = lambda x: {"weight":1.5, 
                                       'color':'green',
                                       'fillColor':'green', 
                                       'fillOpacity':0.25
               },
           
               highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.5, 
                                'weight': 0.1
               },
               
               tooltip=fl.GeoJsonTooltip(
                   fields=['NAME','District'],
                   aliases=['Field:','District:'],
                   labels=True,
                   localize=True
               ),
               ).add_to(california_canalmap)


In [ ]:
# Add facility data
fl.GeoJson(facilities_gdf, name='Facilities', control=True,

           style_function = lambda x: {"weight":1.5, 
                                       'color':'red',
                                       'fillColor':'red',
#                            'fillColor':colormap_counties(x['properties']['LABEL']), 
                            'fillOpacity':0.50
               },
           
               highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.75, 
                                'weight': 0.1
               },
               
               tooltip=fl.GeoJsonTooltip(
                   fields=['Facility','Operator','Production','District'],
                   aliases=['Facility:','Operator:','Production:','District"'],
                   labels=True,
                   localize=True
               ),
               ).add_to(california_canalmap)



In [ ]:
landuse_gdf.head(10)

In [ ]:
# Add landuse data
#for COUNTY in so_cal_counties:

#    print(COUNTY)
    
#fl.GeoJson(landuse_gdf, name='Land Type', control=True, show=False,
fl.GeoJson(landtype_gdf, name='Land Type', control=True, show=False,
           style_function = lambda x: {"weight":1.5, 
                                       'color':'black',
#                                       'fillColor':'grey',
                            'fillColor':colormap_landuse(x['properties']['Use_Index']), 
                            'fillOpacity':0.75
               },
           
               highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.75, 
                                'weight': 0.1
               },
               
               tooltip=fl.GeoJsonTooltip(
#                   fields=['SYMB_CLASS','Crop'],
#                   aliases=['Land Type:','Crop:'],
                   fields=['SYMB_CLASS'],
                   aliases=['Land Type:'],
                   labels=True,
                   localize=True
               ),
               ).add_to(california_canalmap)

In [ ]:
#add individual well data with FastMarkerCluster

zoom_max = 12

#well_points = FastMarkerCluster(wells_latlon,name='Wells',overlay=True,control=True,show=False,
#                               disableClusteringAtZoom=zoom_max).add_to(california_canalmap)

well_points = MarkerCluster(name='Oil & Gas Wells',overlay=True,control=True,show=False,
                               disableClusteringAtZoom=zoom_max).add_to(california_canalmap)    
    
# Note that CircleMarkers are saved to the well_points layer, not the map
wells_gdf.apply(lambda row:
                        fl.CircleMarker(zindexoffset=500,
                                  location=[row['geometry'].y, row['geometry'].x],
                                  max_width=1200,min_width=300,
                                  radius=5,
                                  color='red',
                                  fill=True,
                                  fill_color='orange',
                            
               highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.75, 
                                'weight': 0.1
               },
               
               tooltip = "Operator: %s<br>Well Type: %s<br>District: %s" % (row['OperatorNa'],row['WellTypeLa'],row['District'])             

                                 ).add_to(well_points), 
                         axis=1)

In [ ]:
no_canals = ['Ventura']

if county in no_canals:
    print('No canals in ',county)
else:
    # Add canal data
    fl.GeoJson(canals_gdf, name='Canals and Aqueducts', control=True,overlay=True,

           style_function = lambda x: {"weight":5, 
                                       'color':'blue',
                                       'fillColor':'blue',
#                            'fillColor':colormap_counties(x['properties']['LABEL']), 
                            'fillOpacity':0.25
               },
           
               highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.75, 
                                'weight': 0.1
               },
               
               tooltip=fl.GeoJsonTooltip(
                   fields=['Name','Operator'],
                   aliases=['Facility:','Operator:'],
                   labels=True,
                   localize=True
               ),
               
               ).add_to(california_canalmap)



In [ ]:
# Add minimap
MiniMap(tile_layer='stamenterrain',zoom_level_offset=-4).add_to(california_canalmap)

#colormap_wells.add_to(california_canalmap)

# Finally add control layer and display map
fl.LayerControl(collapsed=False).add_to(california_canalmap)

california_canalmap

In [ ]:
#save the map to an HTML file

if os.path.exists('california_canalmap.html'):
    os.remove('california_canalmap.html')
    
california_canalmap.save('california_canalmap.html')